In [68]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 9999


In [67]:
#24subailes로 모든 order처리 가능
file_list=os.listdir('./dataset/Orderlinelist')
for f in file_list:
    orderlinelist=pd.read_csv('./dataset/Orderlinelist/'+f,delimiter='\t',header=None,engine='python')
    orderlinelist.columns=orderlinelist_column
    if orderlinelist['LocationID'].max()>2881:
        print(f)
    

In [ ]:
path='./dataset'

dist_matrix=pd.read_csv(path+'/Distance matrix/DistanceMatrix_24SubAisles.txt',delimiter='\t',header=None,engine='python')
dist_matrix=dist_matrix.loc[:,:2880]

#orderlist
orderlist_clolumn=['OrderID', 'NumberOfOrderLines', 'ReleaseTime', 'DueTime' ,'FirstOrderLineID']

#orderline
orderlinelist_column=['OrderID', 'OrderLineID', 'AisleID' ,'CellID', 'LevelID', 'LocationID']


orderlist=pd.read_csv(path+'/Orderlist/OrderList_LargeProblems_1_21.txt',delimiter='\t',header=None,engine='python')
orderlist.columns=orderlist_clolumn


orderlinelist=pd.read_csv(path+'/Orderlinelist/OrderLineList_LargeProblems_1_21.txt',delimiter='\t',header=None,engine='python')
orderlinelist.columns=orderlinelist_column



In [86]:
print('distance matrix size:',dist_matrix.to_numpy().shape)
print('orderlist size:','(',len(orderlist),',',len(orderlist.columns),')')
print('orderlinelist size:','(',len(orderlinelist),',',len(orderlinelist.columns),')',end='\n\n')

print('@ orderlist info @')
display(orderlist.info())
print('@ orderlinelist info @')
display(orderlinelist.info())


distance matrix size: (2881, 2881)
orderlist size: ( 595 , 5 )
orderlinelist size: ( 2484 , 6 )

@ orderlist info @
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   OrderID             595 non-null    int64
 1   NumberOfOrderLines  595 non-null    int64
 2   ReleaseTime         595 non-null    int64
 3   DueTime             595 non-null    int64
 4   FirstOrderLineID    595 non-null    int64
dtypes: int64(5)
memory usage: 23.4 KB


None

@ orderlinelist info @
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   OrderID      2484 non-null   int64
 1   OrderLineID  2484 non-null   int64
 2   AisleID      2484 non-null   int64
 3   CellID       2484 non-null   int64
 4   LevelID      2484 non-null   int64
 5   LocationID   2484 non-null   int64
dtypes: int64(6)
memory usage: 116.6 KB


None